In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.optim import Adam

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [4]:
train_dataset = torchvision.datasets.MNIST(root='D:/py-py-py-pytorch/gnn/data',train=True,download=True,
                           transform=transforms.Compose([transforms.ToTensor()]))
train_dataloader = DataLoader(train_dataset,batch_size = 16,shuffle=True)

In [5]:
test_dataset = torchvision.datasets.MNIST(root='D:/py-py-py-pytorch/gnn/data',train=False,download=True,
                           transform=transforms.Compose([transforms.ToTensor()]))
test_dataloader = DataLoader(test_dataset,batch_size = 16,shuffle=True)

In [6]:
it = iter(train_dataloader)
next(it)[0].shape

torch.Size([16, 1, 28, 28])

In [7]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork,self).__init__()
        # [16,1,28,28] => [16,16,26,26]
        self.conv_1 = nn.Conv2d(in_channels=1,out_channels=16,kernel_size=3) #
        # after flatten, size is [16,16*26*26]
        self.lin_1 = nn.Linear(16*26*26,128)
        self.lin_2 = nn.Linear(128,10)

    def forward(self,x):
        # [16,1,28,28] => [16,16,26,26]
        x = self.conv_1(x)
        x = F.relu(x)
        #16x(16x26x26)
        x = x.flatten(start_dim=1)

        x = self.lin_1(x)
        x = F.relu(x)

        logits = self.lin_2(x)
        out = F.softmax(logits,dim=1)
        return out

In [15]:
# t = torch.rand(32,32,26,26)
# t = t.flatten(start_dim=1)
# t.shape

torch.Size([32, 21632])

In [8]:
# Params
learning_rate = 3e-4
num_epochs = 5
model = NeuralNetwork()
model = model.to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(),lr=learning_rate)

In [9]:
# training
for epoch in range(num_epochs):
    train_acc = 0.0
    train_loss = 0.0

    for i, (img,label) in enumerate(train_dataloader):
        imgs = img.to(device)
        labels = label.to(device)

        #forward
        preds = model(imgs)
        #loss
        loss = loss_function(preds,labels)
        #backprop
        optimizer.zero_grad()
        loss.backward()

        #update wts
        optimizer.step()

        train_loss += loss.detach().item()
        train_acc += (torch.argmax(preds,1).flatten() == labels).type(torch.float32).mean().item()

    print("Epoch: %d || Loss: %.2f || Train Acc: %.2f" %(epoch,train_loss/i ,train_acc/i))


Epoch: 0 || Loss: 1.58 || Train Acc: 0.89
Epoch: 1 || Loss: 1.52 || Train Acc: 0.94
Epoch: 2 || Loss: 1.51 || Train Acc: 0.96
Epoch: 3 || Loss: 1.49 || Train Acc: 0.97
Epoch: 4 || Loss: 1.49 || Train Acc: 0.98


In [79]:
x = torch.tensor([1,2,3,4,5,6,7])
y = torch.tensor([1,2,3,8,5,6,9])
val = (x == y).type(torch.float32).mean().item()
print("%.2f" % (val*100))



71.43


In [ ]:
#evaluation
with torch.no_grad():
    test_acc = 0.0
    for i, (imgs,labels) in enumerate(test_dataloader,1):
        imgs = imgs.to(device)
        labels = labels.to(device)
        outs = model(imgs)
        test_acc += (torch.argmax(outs,1).flatten() == labels).type(torch.float32).mean().item()
    
print(test_acc/i)

In [70]:
final_acc = (test_acc/i)*100
print("The test accuracy is %.2f%% " % (final_acc))

The test accuracy is 97.61% 


# Graph Neural Network